# Iniciando análise unitária
### Anaĺise baseado em um único paciente

In [126]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import os

PATH = os.path.abspath('../../')

df = pd.read_csv(PATH+'/data/Generalized_seizures_dataset.csv')

## Estratégia para pegar dados de pacientes individualmente

### Decompondo path

In [127]:
parts = [path.split("/") for path in df['path'].to_list()]

patients = []
sessions = []
files = []

for item in df['path'].to_list():
    part = item.split("/")
    patients.append(part[-3])
    sessions.append(part[-2])
    files.append(part[-1].replace('tse','edf'))

df['patients'] = patients
df['sessions'] = sessions
df['files'] = files

In [128]:
unique_patients = np.unique(patients)

### Carregando dados de paciente

In [129]:
import mne 

# Função para carregar edf's
def load_edf(path):
    path_folder = '/media/davi/2526467b-9f07-4ce5-9425-6d2b458567b7/home/davi/www.isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_seizure/v1.5.2/edf/'
    file = path.replace('tse', 'edf')
    data = mne.io.read_raw_edf(path_folder + file)
    info = data.info
    return {"info": info, "data": data, "file": file.split('/')[-1]}


In [130]:
paciente = unique_patients[0]

data_paciente = df[df['patients'] == paciente].reset_index(drop=True)

In [ ]:
edfs = Parallel(n_jobs=-1) \
        (delayed(load_edf) \
        (item) for item in data_paciente['path'].unique())

### Organizando dados de paciente

In [135]:
# Freq amostral
freq = np.unique([item["info"]['sfreq'] for item in edfs])[0].astype(int)

# Canais do EDF
ch = np.unique([item["info"]['ch_names'] for item in edfs])

# Canal de ECG
ecg_ch = [i for i in ch if 'EKG' in i][0]

In [164]:
def set_labels(df_, file, info):
    
    fs = int(info['sfreq'])

    periods = df[df['files']==file].loc[:,['inicio','fim']].astype({"inicio":int, "fim": int})
    periods = periods.apply(lambda x: x*fs).reset_index(drop=True)

    # COM CRISE = 1 -> yes
    # SEM CRISE = 0 -> not
    df_['labels'] = np.repeat('not',len(df_))

    for index ,row in periods.iterrows():
        df_.loc[row['inicio']:row['fim'],'labels'] = np.repeat('yes',(row['fim']-row['inicio'])+1)  
    
    return df_

def raw2df(raw, file, info):
    df_ =  raw.to_data_frame()
    return set_labels(df_, file, info)


signals = Parallel(n_jobs=-1) \
                (delayed(raw2df) \
                (edf['data'],edf['file'], edf['info']) for edf in edfs)


In [163]:
np.arange(500,525)

array([500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512,
       513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524])

In [166]:
signals = pd.concat(signals).reset_index(drop=True)

In [167]:
signals[signals['labels']=='yes']

,time,EEG FP1-LE,EEG FP2-LE,EEG F3-LE,EEG F4-LE,EEG C3-LE,EEG C4-LE,EEG A1-LE,EEG A2-LE,EEG P3-LE,...,EEG EKG-LE,EEG SP2-LE,EEG SP1-LE,EEG 28-LE,EEG 29-LE,EEG 30-LE,EEG T1-LE,EEG T2-LE,PHOTIC PH,labels
241250,965000,-22.277844,-57.678253,-29.907242,-32.043474,-44.250511,-66.223179,-25.329603,-22.277844,-49.743678,...,-27.465835,-38.757344,-217.590445,-57.678253,-18.615732,-20.446788,-15.258797,-14.038093,0.0,yes
241251,965004,-23.803723,-57.067901,-32.959001,-33.874529,-46.386743,-65.612827,-26.855483,-21.057140,-49.743678,...,-25.939955,-42.114280,-435.486065,-57.678253,-20.446788,-28.991714,-16.784677,-15.563973,0.0,yes
241252,965008,-18.310556,-60.730012,-19.226084,-32.653825,-40.283224,-65.002475,-22.277844,-25.329603,-45.471215,...,-11.901862,-39.367696,-443.420640,-59.814484,-15.869149,-27.465835,-10.681158,-17.700204,0.0,yes
241253,965012,-16.174325,-59.509308,-17.700204,-34.790057,-40.283224,-64.392123,-20.446788,-26.855483,-45.166039,...,-5.493167,-38.452168,-238.037233,-57.067901,-14.343269,-17.700204,-9.155278,-17.089853,0.0,yes
241254,965016,-22.277844,-59.509308,-32.348650,-34.790057,-46.691919,-67.138707,-26.855483,-20.751964,-51.269558,...,-13.732917,-40.283224,-205.993759,-59.204132,-21.057140,-18.310556,-15.258797,-14.953621,0.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218246,1339984,17.395029,1.220704,-24.414075,-16.784677,-26.550307,-44.250511,-8.239750,-26.550307,-27.771010,...,12.207038,-36.010761,-73.547401,0.000000,4.882815,-26.245131,6.103519,22.583020,0.0,yes
1218247,1339988,16.784677,-3.356935,-22.583020,-18.920908,-18.920908,-44.250511,-9.460454,-27.160659,-26.245131,...,35.705585,-35.400409,674.438826,7.019047,4.882815,-18.005380,8.239750,28.381362,0.0,yes
1218248,1339992,0.305176,-13.732917,-35.705585,-26.550307,-39.367696,-55.542021,-14.648445,-23.498547,-47.607447,...,18.005380,-35.400409,955.200690,62.255892,-5.187991,-14.953621,1.220704,22.888195,0.0,yes
1218249,1339996,-1.525880,-16.174325,-38.757344,-29.907242,-50.659206,-60.424836,-11.291510,-24.719251,-54.931669,...,3.662111,-35.400409,264.587539,57.678253,-11.901862,-23.193371,-1.220704,11.291510,0.0,yes


In [80]:
a = pd.concat(signals).reset_index(drop=True)

In [81]:
a[a['time']==0]

,time,EEG FP1-LE,EEG FP2-LE,EEG F3-LE,EEG F4-LE,EEG C3-LE,EEG C4-LE,EEG A1-LE,EEG A2-LE,EEG P3-LE,...,EEG PG2-LE,EEG EKG-LE,EEG SP2-LE,EEG SP1-LE,EEG 28-LE,EEG 29-LE,EEG 30-LE,EEG T1-LE,EEG T2-LE,PHOTIC PH
0,0,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,...,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-24.871839,-163000000.0
340000,0,-5.493167,-30.822770,-21.667492,-16.174325,-30.212418,-35.705585,-3.662111,-30.212418,-15.258797,...,-915.527818,-209.961046,-37.841816,-741.882708,-65.002475,-20.446788,-20.446788,7.629398,-14.343269,0.0
455750,0,-26.855483,-48.217798,-36.010761,-32.348650,-38.757344,-55.847197,-17.395029,-14.953621,-43.029807,...,-991.516626,-5.798343,-39.978048,414.428925,-41.503928,-37.841816,-20.446788,-10.681158,-16.784677,0.0
578750,0,-20.751964,-40.893576,-33.569353,-27.160659,-13.122565,-47.302271,-11.596686,-25.024427,-26.855483,...,1094.360918,-1.220704,-35.705585,-503.845476,-31.738298,-34.484881,-19.226084,-8.850102,-10.070806,0.0
744750,0,-18.920908,-38.146992,-32.348650,-25.329603,-38.146992,-46.386743,-11.291510,-25.939955,-32.043474,...,-1634.827506,14.038093,-36.010761,378.112989,-68.359410,-16.784677,32.043474,6.713871,-21.667492,0.0
883250,0,-12.207038,-30.212418,-21.057140,-25.939955,19.836436,-42.724631,2.136232,-37.841816,-12.207038,...,56.762725,43.640159,3044.129994,2697.755303,-100.708060,0.305176,3351.442164,20.446788,-19.531260,0.0
